<a href="https://colab.research.google.com/github/benjaminbrown038/Hugging-Face/blob/main/object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Hugging Face Object Detection Notebook

[Imports](#imports)

[Data](#data)

In [7]:
%%capture
!pip3 install datasets transformers evaluate timm albumentations
from huggingface_hub import notebook_login
from datasets import load_dataset
import numpy as np
import os 
from PIL import Image, ImageDraw
from transformers import AutoImageProcessor

#notebook_login()

##### Imports 
<a name="imports"></a>

#### Food Data 

##### Data
<a name="data"></a>

In [3]:
cppe5 = load_dataset("cppe-5")

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/29 [00:00<?, ? examples/s]

Dataset cppe-5 downloaded and prepared to /root/.cache/huggingface/datasets/cppe-5/default/1.0.0/dd60c7c8210a67663b06108fb9e23c70acb98e2d3a4f3636f429509b19b74989. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
cppe5

DatasetDict({
    train: Dataset({
        features: ['image_id', 'image', 'width', 'height', 'objects'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['image_id', 'image', 'width', 'height', 'objects'],
        num_rows: 29
    })
})

In [5]:
cppe5["train"][0]

{'image_id': 15,
 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=943x663 at 0x7F0F6E3BABC0>,
 'width': 943,
 'height': 663,
 'objects': {'id': [114, 115, 116, 117],
  'area': [3796, 1596, 152768, 81002],
  'bbox': [[302.0, 109.0, 73.0, 52.0],
   [810.0, 100.0, 57.0, 28.0],
   [160.0, 31.0, 248.0, 616.0],
   [741.0, 68.0, 202.0, 401.0]],
  'category': [4, 4, 0, 0]}}

In [ ]:
image = cppe5["train"][0]["image"]
annotations = cppe5["train"][0]["objects"]
draw = ImageDraw.Draw(image)
categories = cppe5["train"].features["objects"].feature["category"].names
id2label = {index: x for index,x in enumerate(categories,start=0)}
label2id = {v: k for k,v in id2label.items()}
for i in range(len(annotations["id"])):
    box = annotations["bbox"][i-1]
    class_idx = annotations["category"][i-1]
    x,y,w,h = tuple(box)
    draw.rectangle((x,y,x+w,y+h), outline = "red", width = 1)
    draw.text((x,y),id2label[class_idx],fill = "white")

image

remove_idx
keep = [i for i in range(len(cppe5["train"])) if i not in remove_idx]
cppe5["train"] = cppe5["train"].select(keep)


In [ ]:
checkpoint = "facebook/detr-resnet-50"
image_processor = AutoImageProcessor.from_pretrained(checkpoint)
